# 营收预估案例

## 前置说明

### 环境说明

conda版本：https://mirrors.tuna.tsinghua.edu.cn/anaconda/archive/Anaconda3-2021.11-Windows-x86_64.exe


python核心包：
- numpy_financial==1.0.0
- xlsxwriter==3.0.1

### 业务说明

老板有未来几年公司计划的营收目标，想让你拆解成月度目标，自然想到：
- 下月营收 = 上月营收 * （1 + 增速）
- 每年的总目标不同，年间的增速可以不同，年内可以相同

多说两句：
- 强调这只是众多预估方法中的一种，这种方法最平滑
- 强调教学只是展示最小可实现功能的逻辑，实际情况大家根据业务需要去修改

## 参数设置

In [1]:
# 当前每月营收
current_month_list = [
    ['2020-01', 5000],
    ['2020-02', 5500],
    ['2020-03', 5200],
]

In [2]:
# 目标全年营收
target_year_list = [
    ['2020', 80000],
    ['2021', 120000],
    ['2022', 150000],
]
target_year_dict = {k:v for k,v in target_year_list}

In [3]:
target_year_dict

{'2020': 80000, '2021': 120000, '2022': 150000}

In [4]:
# 展示的开始结束时间
start_date = '2020-01'
end_date = '2022-12'

## 计算目标年份的增长率

### 计算当年剩余

In [5]:
import pandas as pd
import numpy as np

In [6]:
current_month_df = pd.DataFrame(current_month_list)

In [8]:
current_month_df.columns = ['month', 'revenue']
current_month_df = current_month_df.sort_values(by='month')
current_month_df

,month,revenue
0,2020-01,5000
1,2020-02,5500
2,2020-03,5200


In [9]:
# 生成年份
current_month_df['year'] = current_month_df['month'].apply(lambda x: x[:4])
current_month_df

,month,revenue,year
0,2020-01,5000,2020
1,2020-02,5500,2020
2,2020-03,5200,2020


In [12]:
# 计算当前真实数据中的全年发生，这里计数、金额合计都需要
current_group_df = current_month_df.groupby('year').agg({'month': 'count', 'revenue': 'sum'})
current_group_df

,month,revenue
year,,
2020,3,15700


In [13]:
# 当前年份
this_year = current_group_df.index[0]
# 获取当前年份的目标
this_year_target = target_year_dict[this_year]
# 当年剩余营收目标
this_year_residual = this_year_target - current_group_df['revenue'][0]
# 当前剩余月份
this_year_month_past = current_group_df['month'][0]
this_year_month_residual = 12 - this_year_month_past
# 查看结果
this_year, this_year_target, this_year_residual, this_year_month_past, this_year_month_residual

('2020', 80000, 64300, 3, 9)

In [18]:
# 假设增速年内每月固定增长r，真实的末月为x，预估的首月为x*(1+r)，次月为x*(1+r)**2... 累计所有和全年剩余目标接近即可
this_year_actual_last = current_month_df[current_month_df['year'] == this_year]['revenue'].iloc[-1]
this_year_actual_last

5200

### 计算等比数列r

$$f(r) = Revenue*(1+r) +  Revenue*(1+r) ^2  + \dots +  Revenue*(1+r) ^9 = Residual\_value$$

$if: r_1 < r_2, f(r_1) < Residual\_value, f(r_2) > Residual\_value，则真正的r \in (r_1, r_2)$

#### 法1：暴力精度

In [21]:
# 假设比例在 0.1~1.0 之间，每次移动 0.1
# 这里用range报错，所以才用np.arange
this_year_r_pred_list = np.arange(0.1, 1.1, 0.1)

In [24]:
this_year_actual_last

5200

In [25]:
this_year_month_residual

9

In [26]:
# 暴力循环
last_r, last_total = 0, 0
for r in this_year_r_pred_list:
    # 计算今年剩余月份的总和
    # TODO 优化，等比数列
    tmp_total = 0
    for i in range(this_year_month_residual):
        tmp_total += this_year_actual_last*(1+r)**(i+1)
    if last_total < this_year_residual and tmp_total > this_year_residual:
        print(f'r between {last_r}~{r}, {r} total value is {tmp_total} more than resisual {this_year_residual}')
        break
    last_r = r
    last_total = tmp_total

r between 0~0.1, 0.1 total value is 77674.60792520003 more than resisual 64300


In [27]:
# 精度加10倍
this_year_r_pred_list = np.arange(0.01, 0.1, 0.01)
# 暴力循环
last_r, last_total = 0, 0
for r in this_year_r_pred_list:
    # 计算今年剩余月份的总和
    # TODO 优化，等比数列
    tmp_total = 0
    for i in range(this_year_month_residual):
        tmp_total += this_year_actual_last*(1+r)**(i+1)
    if last_total < this_year_residual and tmp_total > this_year_residual:
        print(f'r between {last_r}~{r}, {r} total value is {tmp_total} more than resisual {this_year_residual}')
        break
    last_r = r
    last_total = tmp_total

r between 0.060000000000000005~0.06999999999999999, 0.06999999999999999 total value is 66645.52939865344 more than resisual 64300


##### 指定精度暴力求解

In [40]:
# 精度控制，增速保留几位小数
r_precision = 6
# 控制增速上限，不可能1个月做到上个月10倍吧
r_up_limit = 1

In [41]:
# 根据精度来
this_year_r_pred_list = np.arange(0.1**r_precision, r_up_limit, 0.1**r_precision)

In [42]:
this_year_r_pred_list

array([1.00000e-06, 2.00000e-06, 3.00000e-06, ..., 9.99997e-01,
       9.99998e-01, 9.99999e-01])

In [43]:
# 暴力循环
last_r, last_total = 0, 0
for r in this_year_r_pred_list:
    # 计算今年剩余月份的总和
    # TODO 优化，等比数列
    tmp_total = 0
    for i in range(this_year_month_residual):
        tmp_total += this_year_actual_last*(1+r)**(i+1)
    if last_total < this_year_residual and tmp_total > this_year_residual:
        print(f'r between {last_r}~{r}, {r} total value is {tmp_total} more than resisual {this_year_residual}')
        break
    last_r = r
    last_total = tmp_total

r between 0.06295800000000003~0.06295900000000003, 0.06295900000000003 total value is 64300.067043483854 more than resisual 64300


#### 法2：IRR

##### 什么是NPV和IRR？

货币是有时间价值的

代入到一个商业住宅的项目，第一年我要去建这个写字楼，之后每年收租，那么第一期的现金流CF0是一个巨额负数，也可以称之为-Investment，之后每期CF1、CF2都是正数，而折现率是我做其他事情的**机会成本**，比如如果我不做这个项目，我的钱也是可以产生价值的，我可以全部买收益率为2%无风险的国债，那么每年我都能收到国债的本息，这种过程也可以称为**货币的时间价值的体现**。由于每期我都可以投资国债，所以我每期都要用现金流除以对应期数的国债收益率。最终我将每期的现金流加和就形成了**NPV（净现值，Net Present Value）**，如果NPV大于0，那么这个项目是可以投资的，最起码不会比直接买国债赚的少。

这种计算NPV的方法我们一般称为**折现现金流模型（Discounted Cash Flow，DCF）**，为了计算投资一个项目的收益率情况，我们可以先将折现率设定为一个未知数R，让R不断变动，直到让NPV等于0，此时的R就能描述出投资这个项目的平均年化收益，我们也将这个R称为**内部收益率IRR**（Internal Rate of Return），这个收益率是假设了你产生的营收可以进行再投资，因此收益率会更高

$$NPV = -Investment + \frac{CF1}{(1+R)} + \frac{CF2}{(1+R)^2}  + \dots + \frac{CFn}{(1+R)^n}$$

$$0 = -Investment + \frac{CF1}{(1+IRR)} + \frac{CF2}{(1+IRR)^2}  + \dots + \frac{CFn}{(1+IRR)^n}$$

$$0 = -Residual\_value + \frac{Revenue}{1/(1+r)} + \frac{Revenue}{1/(1+r)^2}  + \dots + \frac{Revenue}{1/(1+r)^9}$$

##### 聪明的使用数学法

In [46]:
irr_in_list = [this_year_actual_last for _ in range(this_year_month_residual)]
irr_total_list = [-this_year_residual] + irr_in_list

In [47]:
irr_total_list

[-64300, 5200, 5200, 5200, 5200, 5200, 5200, 5200, 5200, 5200]

In [49]:
import numpy_financial as npf
irr = npf.irr(irr_total_list)

In [51]:
irr2rate = 1 / (1 + irr) - 1 
irr2rate

0.06295879494637924

In [52]:
irr_total = 0
for i in range(this_year_month_residual):
    irr_total += this_year_actual_last*(1+irr2rate)**(i+1)
    
irr_total, this_year_residual

(64300.000000000044, 64300)

## 通用计算年份增长率

In [54]:
target_year_df = pd.DataFrame(target_year_list)
target_year_df.columns = ['year', 'revenue']
target_year_df = target_year_df.sort_values(by='year')
target_year_df

,year,revenue
0,2020,80000
1,2021,120000
2,2022,150000


In [55]:
target_year_df['completed'] = 0
target_year_df['residual_month'] = 12
target_year_df['last_month_revenue'] = 0

In [56]:
target_year_df

,year,revenue,completed,residual_month,last_month_revenue
0,2020,80000,0,12,0
1,2021,120000,0,12,0
2,2022,150000,0,12,0


In [62]:
# 给已完成的年份赋值
target_year_df = target_year_df.set_index('year')

In [60]:
target_year_df.loc[this_year, 'completed'] = this_year_target - this_year_residual
target_year_df.loc[this_year, 'residual_month'] = this_year_month_residual
target_year_df.loc[this_year, 'last_month_revenue'] = this_year_actual_last

In [64]:
target_year_df = target_year_df.reset_index()

In [68]:
target_year_df

,year,revenue,completed,residual_month,last_month_revenue
0,2020,80000,15700,9,5200
1,2021,120000,0,12,0
2,2022,150000,0,12,0


In [69]:
import numpy_financial as npf

every_irr_list = []  # 存储每年计算月增速
last_month_revenue_list = []  # 存储每年末月收入

for i in target_year_df.index:
    # 取数
    tmp_revenue = target_year_df.loc[i, 'revenue'] 
    tmp_completed = target_year_df.loc[i, 'completed'] 
    tmp_residual_month = target_year_df.loc[i, 'residual_month'] 
    tmp_last_month_revenue = target_year_df.loc[i, 'last_month_revenue'] 
    # 获取上次最末月的值
    if not last_month_revenue_list:  # 如果没存的话，就用df的
        last_month_revenue_list.append(tmp_last_month_revenue)
    tmp_last_month_revenue = last_month_revenue_list[-1]
    # 计算irr
    tmp_irr_in_list = [tmp_last_month_revenue for _ in range(tmp_residual_month)]
    tmp_irr_total_list = [-(tmp_revenue - tmp_completed)] + tmp_irr_in_list
    tmp_irr = npf.irr(tmp_irr_total_list)
    tmp_irr2rate = 1 / (1 + tmp_irr) - 1 
    every_irr_list.append(tmp_irr2rate)
    # 给末月赋值用于下次迭代
    iter_last_revenue = tmp_last_month_revenue * (1+tmp_irr2rate) ** (tmp_residual_month)
    last_month_revenue_list.append(iter_last_revenue)  # 再存一次

In [70]:
every_irr_list, last_month_revenue_list

([0.06295879494637924, 0.015960789453421054, 0.020982943947220667],
 [5200, 9008.473606219515, 10893.67897466778, 13976.43526432213])

In [71]:
target_year_df['mom'] = every_irr_list
target_year_df['last_month_revenue'] = last_month_revenue_list[:-1]

In [72]:
target_year_df

,year,revenue,completed,residual_month,last_month_revenue,mom
0,2020,80000,15700,9,5200.000000,0.062959
1,2021,120000,0,12,9008.473606,0.015961
2,2022,150000,0,12,10893.678975,0.020983


## 对年份进行炸裂-explode

### 什么是炸裂

当前的数据粒度不同，一个是月份、一个是年份，我们学过将月->年，那么如何将年->月呢？需要将年份炸裂！

In [73]:
test_df = pd.DataFrame([['a', [60, 50, 50]], ['b', [50, 40]]])
test_df.columns = ['姓名', '健身时长']
test_df

,姓名,健身时长
0,a,"[60, 50, 50]"
1,b,"[50, 40]"


In [74]:
test_df.explode('健身时长')

,姓名,健身时长
0,a,60
0,a,50
0,a,50
1,b,50
1,b,40


### 对年份炸裂

In [75]:
target_year_df['residual_month_list'] = target_year_df['residual_month'].apply(lambda x : [12-x+i+1 for i in range(x)])
target_year_df

,year,revenue,completed,residual_month,last_month_revenue,mom,residual_month_list
0,2020,80000,15700,9,5200.000000,0.062959,"[4, 5, 6, 7, 8, 9, 10, 11, 12]"
1,2021,120000,0,12,9008.473606,0.015961,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]"
2,2022,150000,0,12,10893.678975,0.020983,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]"


In [ ]:
target_month_df = target_year_df.explode('residual_month_list')
target_month_df

In [ ]:
# 使用累乘的方法
mom_cumprod_se = (target_month_df['mom'] + 1).cumprod()
mom_cumprod_se

In [81]:
target_month_df['revenue'] = target_month_df['last_month_revenue'].iloc[0] * mom_cumprod_se

In [84]:
# 构造日期
target_month_df['month'] = target_month_df['residual_month_list'].astype(str).apply(lambda x : x if len(x) == 2 else '0'+x)
target_month_df['month'] = target_month_df['year'] + '-' + target_month_df['month']

In [86]:
# 拼接前后
pred_month_df = target_month_df.loc[:, ['month', 'revenue']]
pred_month_df['type'] = 'E'

In [87]:
actual_month_df = current_month_df.loc[:, ['month', 'revenue']]
actual_month_df['type'] = 'A'

In [90]:
# 构造营收预估数据
total_month_df = pd.concat([actual_month_df, pred_month_df], axis=0).sort_values(by='month')

In [91]:
total_month_df.columns = ['月份', '营收（万元）', '类型']
total_month_df

,月份,营收（万元）,类型
0,2020-01,5000.000000,A
1,2020-02,5500.000000,A
2,2020-03,5200.000000,A
0,2020-04,5527.385734,E
0,2020-05,5875.383279,E
0,2020-06,6245.290330,E
0,2020-07,6638.486283,E
0,2020-08,7056.437380,E
0,2020-09,7500.702174,E
0,2020-10,7972.937344,E


In [92]:
total_month_df.index = range(total_month_df.shape[0])
total_month_df.head(5)

,月份,营收（万元）,类型
0,2020-01,5000.000000,A
1,2020-02,5500.000000,A
2,2020-03,5200.000000,A
3,2020-04,5527.385734,E
4,2020-05,5875.383279,E


## 模板导出

官网：https://xlsxwriter.readthedocs.io/  
普通案例：https://xlsxwriter.readthedocs.io/examples.html  
图表案例：https://xlsxwriter.readthedocs.io/chart_examples.html  
样式调整：https://xlsxwriter.readthedocs.io/format.html#format

### 表格创建

In [124]:
import xlsxwriter as xw

In [125]:
xw.__version__

'3.0.1'

In [126]:
# 创建新的excel
workbook = xw.Workbook('./营收规划.xlsx')
# 添加sheet
worksheet = workbook.add_worksheet('每月营收预估')

### 单元格写入

In [127]:
## 一个单元格一个格子去写
# 写法1：传统Excel
worksheet.write('A1', '月份')
worksheet.write('B1', '营收（万元）')
# 写法2：行-列索引
worksheet.write(0, 2, '类型')  # C1

0

In [128]:
# 数字转字母
from xlsxwriter.utility import xl_rowcol_to_cell, xl_cell_to_rowcol
xl_rowcol_to_cell(0, 100) # CW1

'CW1'

In [129]:
# 按行写入
for row in range(total_month_df.shape[0]):
    worksheet.write_row(row+1, 0, total_month_df.iloc[row].values)
# 按列写入更好，可以改数据格式：https://xlsxwriter.readthedocs.io/format.html#format

### 公式写入

In [130]:
## 最下面增加一个合计项目
# 获取末尾行
last_row = total_month_df.shape[0] + 1 # 最后一行
# 格式设置
bold = workbook.add_format({'bold': True, 'font_color': '#000000'})
worksheet.write(last_row, 0, '合计', bold)
# 公式写入
worksheet.write(last_row, 1, f'=SUM(B2:B{last_row})', bold)

0

### 图表写入

In [131]:
# 折线图
chart = workbook.add_chart({'type': 'line'})

In [132]:
chart.add_series({
    'categories': f'=每月营收预估!A2:A{last_row}',  # 横坐标
    'values': f'=每月营收预估!B2:B{last_row}',  # 数值
    'name': '=每月营收预估!B1',  # 列名
    'marker': {'type': 'diamond'},  # 增加标记
})
chart.set_title({'name': '每月营收走势图'})  # 设置标题
# 插入表格
worksheet.insert_chart('D1', chart)

0

In [133]:
# 增加框线
cell_format = workbook.add_format({'border':1})
worksheet.conditional_format('A1:XFD1048576', {'type': 'no_blanks', 'format': cell_format})

0

In [134]:
## 保存excel
workbook.close()

## 代码封装

In [138]:
def predict_month_revenue(current_month_list, target_year_list, file_name='./营收规划.xlsx'):
    import numpy as np
    import pandas as pd
    import xlsxwriter as xw
    import numpy_financial as npf
    
    target_year_dict = {k:v for k,v in target_year_list}
    ################################################################
    ## 数据读取
    # 读取月份数据
    current_month_df = pd.DataFrame(current_month_list)
    current_month_df.columns = ['month', 'revenue']
    current_month_df = current_month_df.sort_values(by='month')
    # 生成年份
    current_month_df['year'] = current_month_df['month'].apply(lambda x: x[:4])
    # 计算当前真实数据中的全年发生，这里计数、金额合计都需要
    current_group_df = current_month_df.groupby('year').agg({'month': 'count', 'revenue': 'sum'})

    ################################################################
    ## 增速计算准备
    # 当前年份
    this_year = current_group_df.index[0]
    # 获取当前年份的目标
    this_year_target = target_year_dict[this_year]
    # 当年剩余营收目标
    this_year_residual = this_year_target - current_group_df['revenue'][0]
    # 当前剩余月份
    this_year_month_past = current_group_df['month'][0]
    this_year_month_residual = 12 - this_year_month_past
    # 假设增速年内每月固定增长r，真实的末月为x，预估的首月为x*(1+r)，次月为x*(1+r)**2... 累计所有和全年剩余目标接近即可
    this_year_actual_last = current_month_df[current_month_df['year'] == this_year]['revenue'].iloc[-1]

    ################################################################
    # 目标年份构建
    target_year_df = pd.DataFrame(target_year_list)
    target_year_df.columns = ['year', 'revenue']
    target_year_df = target_year_df.sort_values(by='year')
    target_year_df['completed'] = 0
    target_year_df['residual_month'] = 12
    target_year_df['last_month_revenue'] = 0

    # 给已完成的年份赋值
    target_year_df = target_year_df.set_index('year')
    target_year_df.loc[this_year, 'completed'] = this_year_target - this_year_residual
    target_year_df.loc[this_year, 'residual_month'] = this_year_month_residual
    target_year_df.loc[this_year, 'last_month_revenue'] = this_year_actual_last
    target_year_df = target_year_df.reset_index()

    # 增速计算
    every_irr_list = []  # 存储每年计算月增速
    last_month_revenue_list = []  # 存储每年末月收入
    for i in target_year_df.index:
        # 取数
        tmp_revenue = target_year_df.loc[i, 'revenue'] 
        tmp_completed = target_year_df.loc[i, 'completed'] 
        tmp_residual_month = target_year_df.loc[i, 'residual_month'] 
        tmp_last_month_revenue = target_year_df.loc[i, 'last_month_revenue'] 
        # 获取真实上年最末月的值
        if not last_month_revenue_list:  # 如果没存的话，就用df的
            last_month_revenue_list.append(tmp_last_month_revenue)
        tmp_last_month_revenue = last_month_revenue_list[-1]
        # 计算irr
        tmp_irr_in_list = [tmp_last_month_revenue for _ in range(tmp_residual_month)]
        tmp_irr_total_list = [-(tmp_revenue - tmp_completed)] + tmp_irr_in_list
        tmp_irr = npf.irr(tmp_irr_total_list)
        tmp_irr2rate = 1 / (1 + tmp_irr) - 1 
        every_irr_list.append(tmp_irr2rate)
        # 给末月赋值用于下次迭代
        iter_last_revenue = tmp_last_month_revenue * (1+tmp_irr2rate) ** (tmp_residual_month)
        last_month_revenue_list.append(iter_last_revenue)  # 再存一次

    target_year_df['mom'] = every_irr_list
    target_year_df['last_month_revenue'] = last_month_revenue_list[:-1]
    
    ################################################################
    ## 对年度数据炸裂到月度
    target_year_df['residual_month_list'] = target_year_df['residual_month'].apply(lambda x : [12-x+i+1 for i in range(x)])
    target_month_df = target_year_df.explode('residual_month_list')
    # 使用累乘的方法
    mom_cumprod_se = (target_month_df['mom'] + 1).cumprod()
    target_month_df['revenue'] = target_month_df['last_month_revenue'].iloc[0] * mom_cumprod_se
    # 构造日期
    target_month_df['month'] = target_month_df['residual_month_list'].astype(str).apply(lambda x : x if len(x) == 2 else '0'+x)
    target_month_df['month'] = target_month_df['year'] + '-' + target_month_df['month']
    
    ################################################################
    ## 数据拼接导出
    pred_month_df = target_month_df.loc[:, ['month', 'revenue']]
    pred_month_df['type'] = 'E'
    actual_month_df = current_month_df.loc[:, ['month', 'revenue']]
    actual_month_df['type'] = 'A'
    total_month_df = pd.concat([actual_month_df, pred_month_df], axis=0).sort_values(by='month')
    total_month_df.columns = ['月份', '营收（万元）', '类型']
    total_month_df.index = range(total_month_df.shape[0])
    
    ################################################################
    ## 导出excel
    workbook = xw.Workbook(file_name)
    # 添加sheet
    worksheet = workbook.add_worksheet('每月营收预估')
    ## 一个单元格一个格子去写
    # 写法1：传统Excel
    worksheet.write('A1', '月份')
    worksheet.write('B1', '营收（万元）')
    # 写法2：行-列索引
    worksheet.write(0, 2, '类型')  # C1
    # 按行写入
    for row in range(total_month_df.shape[0]):
        worksheet.write_row(row+1, 0, total_month_df.iloc[row].values)

    ## 最下面增加一个合计项目
    # 获取末尾行
    last_row = total_month_df.shape[0] + 1# 最后一行
    # 格式设置
    bold = workbook.add_format({'bold': True, 'font_color': '#000000'})
    worksheet.write(last_row, 0, '合计', bold)
    # 公式写入
    worksheet.write(last_row, 1, f'=SUM(B2:B{last_row})', bold)

    ## 折线图
    chart = workbook.add_chart({'type': 'line'})
    chart.add_series({
        'categories': f'=每月营收预估!A2:A{last_row}',  # 横坐标
        'values': f'=每月营收预估!B2:B{last_row}',  # 数值
        'name': '=每月营收预估!B1',  # 列名
        'marker': {'type': 'diamond'},  # 增加标记
    })
    chart.set_title({'name': '每月营收走势图'})  # 设置标题
    # 插入表格
    worksheet.insert_chart('D1', chart)
    # 增加框线
    cell_format = workbook.add_format({'border':1})
    worksheet.conditional_format('A1:XFD1048576', {'type': 'no_blanks', 'format': cell_format})
    ## 保存excel
    workbook.close()
    print('预估表格生成成功！')

In [139]:
# 当前每月营收
current_month_list = [
    ['2020-01', 5000],
    ['2020-02', 5500],
    ['2020-03', 5200],
    ['2020-04', 5000],
]

In [140]:
# 目标全年营收
target_year_list = [
    ['2020', 80000],
    ['2021', 120000],
    ['2022', 180000],
    ['2023', 300000],
]


In [141]:
predict_month_revenue(current_month_list, target_year_list, '新.xlsx')

预估表格生成成功！
